In [1]:
import findspark
findspark.init('/usr/local/spark')

In [2]:
from pyspark import SparkContext
sc=SparkContext(appName="capstone")


In [ ]:
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *


In [4]:
spark=SparkSession.builder.appName("capstone1").config("spark.sql.warehouse.dir", "hdfs://usr/hive/warehouse").enableHiveSupport().getOrCreate()

In [6]:
data=spark.read.load("data.csv",format="csv",sep=',',header=True,inferSchema=True)
data.printSchema()

root
 |-- CLICK_FLG: string (nullable = true)
 |-- OPEN_FLG: string (nullable = true)
 |-- ADDR_VER_CD: string (nullable = true)
 |-- AQI: string (nullable = true)
 |-- ASIAN_CD: integer (nullable = true)
 |-- AUTO_IN_MARKET: integer (nullable = true)
 |-- BIRD_QTY: integer (nullable = true)
 |-- BUYER_DM_BOOKS: string (nullable = true)
 |-- BUYER_DM_COLLECT_SPC_FOOD: string (nullable = true)
 |-- BUYER_DM_CRAFTS_HOBBI: string (nullable = true)
 |-- BUYER_DM_FEMALE_ORIEN: string (nullable = true)
 |-- BUYER_DM_GARDEN_FARM: string (nullable = true)
 |-- BUYER_DM_GENERAL: string (nullable = true)
 |-- BUYER_DM_GIFT_GADGET: string (nullable = true)
 |-- BUYER_DM_MALE_ORIEN: string (nullable = true)
 |-- BUYER_DM_UPSCALE: string (nullable = true)
 |-- BUYER_MAG_CULINARY_INTERS: string (nullable = true)
 |-- BUYER_MAG_FAMILY_GENERAL: string (nullable = true)
 |-- BUYER_MAG_FEMALE_ORIENTED: string (nullable = true)
 |-- BUYER_MAG_GARDEN_FARMING: string (nullable = true)
 |-- BUYER_MAG_HEALTH

In [7]:
data.select("Mailed_Date").show(1,truncate=False)

+----------------------+
|Mailed_Date           |
+----------------------+
|Wed, 09/29/10 03:52 PM|
+----------------------+
only showing top 1 row



## extracting date value from Mailed_Date column to make queries comaptible and increase query execution performance

In [8]:
data=data.withColumn("Day",substring("Mailed_Date",0,3))
data.select("Day").show(2)

+---+
|Day|
+---+
|Wed|
|Tue|
+---+
only showing top 2 rows



In [9]:
data=data.withColumn("Month",substring("Mailed_Date",6,2).cast(IntegerType()))
data=data.withColumn("DayNum",substring("Mailed_Date",9,2).cast(IntegerType()))
data=data.withColumn("Year",substring("Mailed_Date",12,2).cast(IntegerType()))
data=data.withColumn("Hour",substring("Mailed_Date",15,2).cast(IntegerType()))
data=data.withColumn("Minute",substring("Mailed_Date",18,2).cast(IntegerType()))
data=data.withColumn("AMPM",substring("Mailed_Date",21,2))

In [10]:
data.select(data['Month'],data['DayNum'],data['Year'],data['Hour'],data['Minute'],data['AMPM']).show(2,truncate=False)

+-----+------+----+----+------+----+
|Month|DayNum|Year|Hour|Minute|AMPM|
+-----+------+----+----+------+----+
|9    |29    |10  |3   |52    |PM  |
|9    |21    |10  |11  |19    |AM  |
+-----+------+----+----+------+----+
only showing top 2 rows



In [11]:
data.write.saveAsTable("jigsaw.capstone")

In [12]:
sc.stop()
spark.stop()